# 演習問題1
## Word2Vecを用いて単語の分散表現を得る

下記のコードでエラーが出る場合は，コマンドプロンプトで実行するか，`pip install gensim=4.3.3 --user`に書き換えて実行して下さい．

In [1]:
#pip install gensim=4.3.3

### 使用データセット
### **livedoor ニュースコーパス**
NHN Japan株式会社が運営するlivedoor ニュースの記事から可能な限りHTMLタグを取り除いて作成したもの．<br>
ニュース記事は分野ごとに分けられており，今回は"ITライフハック"についての記事をコーパスに用いる．<br>

参考記事：<a>https://www.rondhuit.com/download.html#ldcc

In [2]:
import numpy as np
import pandas as pd
import MeCab
import ipadic
import re
from sklearn.decomposition import PCA
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams

#rcParamsに文字化けしないようにフォントの設定を行う
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']

In [3]:
# 形態素解析器の初期化
tagger = MeCab.Tagger(ipadic.MECAB_ARGS)
tagger.parse("")  # MeCabの初期化（空文字列を解析することで初期化）

'EOS\n'

In [4]:
# トークナイズ関数
def tokenize(text):
    node = tagger.parseToNode(text)
    words = []
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞", "接頭詞", "動詞", "形容詞", "副詞", "助詞", "助動詞", "連体詞", "接続詞", "感動詞", "記号", "フィラー"]:
            words.append(node.surface)
        node = node.next
    return words

In [5]:
# 文章を入力しトークナイズができるか確認
print(tokenize("ここに文章を入力"))

['ここ', 'に', '文章', 'を', '入力']


In [6]:
# コーパスの読み込み
df = pd.read_csv("./csv/it-life-hack.csv")

# トークン化
sentences = df["body"].apply(tokenize)

### 元文章とトークン化した文章を見比べる

In [7]:
# 元文章を表示
print("元文章：")
print(df["body"][0])

print("")
# トークン化した結果を表示
print("トークン化結果：")
before_sentence = sentences[0]
print(before_sentence)

元文章：
旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック【ITフラッシュバック】
テレビやTwitterと連携できるパソコンや、プロセッサや切り替わるパソコンなど、面白いパソコンが次から次へと登場した。旧式Macの禁断ともいえるパワーアップ方法から、NECの最新PC、話題のThinkPad X1 Hybrid、新セキュリティソフトまで一挙に紹介しよう。

■インテル SSD 520をMacに装着！旧式Macはどれほど高速化するのか (上)
インテルが最新SSD「520シリーズ」を発売した。現行SSDの中でもトップクラスの性能を誇る同製品を、旧型Macの高速化を図るというポイントでレビューしてみた。少し風変わりなレビューとなるが、どの程度の効果があるか、期待大である。


■http://itlifehack.jp/archives/6716997.html
ThinkPad X1 Hybridは使用するCPUがx86(インテルCore iなど)からARMに切り替わるハイブリッドなPCだが、これと同時にOSも切り替わる。


■初期費用、更新費用ともに無料！ジャストシステム、ヤモリが目印のセキュリティソフト
現在では、多くのユーザーがパソコンにセキュリティソフトを導入しているが、その過半数は毎年5,000円程度かかる更新費用やその手続きについて不満を持っている。有料ソフトを利用するユーザーの約8割は無料のセキュリティソフトを知っているにもかかわらず、性能面で劣るのではという不安から導入を控えているという状況にある。


■テレビの新しい楽しみ方を提案！NECの春PCはTVとTwitterの連携
NECは2012年2月14日、個人向けデスクトップパソコン「VALUESTAR」シリーズ3タイプ16モデルを2月16日より販売すると発表した。新商品では、よりパワフルになった録画機能に加え、TV視聴・録画機能に業界で初めて人気のTwitterを連携させた「SmartVisionつぶやきプラス」を追加するなど、TVパソコンならではの機能を搭載。スマートフォン、ホームネットワーク対応も強化し、「安心・簡単・快適」なデジタルエンターテイメントの提案として、主要モデルに対し以下の強化を行った。


■まるでお祭りの出荷式！レッツノートSX1の出荷が始まる
2月24日に

### 問1 Word2Vecを用いてモデルを作成

今回は，パラメータを vector_size=100, window=5, min_count=1, sg=0, seed=42に設定して学習を実行してください．

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=1,
    sg=0,
    seed=42
)

### 問2 任意の単語が持つ単語ベクトルを取得
今回はtarget_wordを"スマホ"とします

In [9]:
# ターゲットワードを設定
target_word = "スマホ"

In [10]:
vec = model.wv[target_word]
print(vec)

[-3.13411146e-01 -1.45405248e-01 -9.13846552e-01 -1.27537906e+00
 -3.45222741e-01  7.55985677e-01  1.92862082e+00  2.14412913e-01
  7.21011609e-02  5.34404591e-02 -5.20982325e-01  5.67540407e-01
  6.38190687e-01  5.19829333e-01 -4.41216499e-01 -9.99293327e-01
 -4.13411111e-01 -7.57146060e-01 -4.28544760e-01  6.04099035e-01
 -3.13185543e-01  1.45269763e+00  1.12880003e+00  7.36420512e-01
  1.06598198e+00  2.16118723e-01 -4.79636490e-01  7.68663883e-01
  9.95072484e-01 -1.62593067e+00  8.15783441e-02  6.53206766e-01
  4.07871567e-02  3.09135523e-02  3.70733291e-01  1.31943181e-01
  2.87614703e-01 -6.31473839e-01  5.36378697e-02 -3.29889596e-01
 -3.29470448e-02 -8.11222196e-01  8.94857526e-01 -8.98034394e-01
 -1.92230716e-01  4.15342376e-02  3.33859146e-01 -6.36302650e-01
  1.80821621e+00  1.21992387e-01 -7.48061240e-01 -2.11037970e+00
  9.76502717e-01  1.78997481e+00  6.64376557e-01  1.04952246e-01
  2.68263817e-01  3.03194433e-01 -4.79248399e-03 -9.73522291e-02
 -1.77259415e-01 -4.96359

### 問3 任意の単語と類似度の高い単語上位10個を表示

In [11]:
# 任意の単語と類似度が高い単語トップ10を表示
print(f"\"{target_word}\"と類似度の高い単語トップ10")
print("単語\t :類似度(最大値1, 最小値-1)")
for ranking in model.wv.most_similar(target_word, topn=10):
    print(f"{ranking[0]}\t :{ranking[1]}")

"スマホ"と類似度の高い単語トップ10
単語	 :類似度(最大値1, 最小値-1)
強い	 :0.8931419253349304
味方	 :0.8449782133102417
兵器	 :0.8356143236160278
スマ	 :0.8265565037727356
スタンド	 :0.8244155645370483
ケース	 :0.8136326670646667
潜入	 :0.8062107563018799
驚愕	 :0.8046680688858032
最新	 :0.7998527884483337
SIII	 :0.7960683107376099


### 問4 任意の2単語間における類似度を表示
今回はtarget_word2を"難しい"とします

In [12]:
# ターゲットワード2を設定
target_word2 = "難しい"

In [13]:
# 任意の2単語の類似度を表示
print(f"\"{target_word}\"と\"{target_word2}\"の類似度")
print(model.wv.similarity(target_word, target_word2))

"スマホ"と"難しい"の類似度
0.32267633


### 問5 任意の2単語のベクトルの差を取り，結果ベクトルの周辺単語を表示
今回は"スマホ"-"難しい"を計算してください

In [14]:
# ベクトルの差を計算
print(f"\"{target_word}\"-\"{target_word2}\"と類似度の高い単語トップ10")
print("単語\t :類似度(最大値1, 最小値-1)")
for ranking in model.wv.most_similar(positive=[target_word], negative=[target_word2]):
    print(f"{ranking[0]}\t :{ranking[1]}")

"スマホ"-"難しい"と類似度の高い単語トップ10
単語	 :類似度(最大値1, 最小値-1)
エヌ・ティ・ティ	 :0.6373079419136047
ファーストイーサネット	 :0.5560752749443054
太刀打ち	 :0.540340006351471
モバイルルーター	 :0.5328551530838013
モニタ	 :0.5227583050727844
si	 :0.518841028213501
タテヨコサイズ	 :0.5143628716468811
HK	 :0.5040014982223511
盤面	 :0.4982357621192932
スマートファミリンク	 :0.4976561665534973


### 問6 学習済みモデルを保存
モデル名は"simple_model.model"にしてください．

In [15]:
model.save("./model/simple_model.model")

## 前処理の導入
ipadeic辞書は品詞の大枠を[名詞, 接頭辞, 動詞, 形容詞, 副詞, 助詞, 助動詞, 連体詞, 接続詞, 感動詞, 記号, フィラー]で分類する．

In [16]:
# ipadeicの辞書が分類する品詞を表示
parsed = tagger.parse("人間 お 働く 忙しい まるで を らしい この そして はぁ ! あのー")
print(parsed)

人間	名詞,一般,*,*,*,*,人間,ニンゲン,ニンゲン
お	接頭詞,名詞接続,*,*,*,*,お,オ,オ
働く	動詞,自立,*,*,五段・カ行イ音便,基本形,働く,ハタラク,ハタラク
忙しい	形容詞,自立,*,*,形容詞・イ段,基本形,忙しい,イソガシイ,イソガシイ
まるで	副詞,一般,*,*,*,*,まるで,マルデ,マルデ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
らしい	助動詞,*,*,*,形容詞・イ段,基本形,らしい,ラシイ,ラシイ
この	連体詞,*,*,*,*,*,この,コノ,コノ
そして	接続詞,*,*,*,*,*,そして,ソシテ,ソシテ
はぁ	感動詞,*,*,*,*,*,はぁ,ハァ,ハー
!	記号,一般,*,*,*,*,*
あのー	フィラー,*,*,*,*,*,あのー,アノー,アノー
EOS



### 問7 品詞, ストップワードの除去
`stop_words.txt`を読み込みストップワード(不要単語)を除去する．<br>
英単語を正規化し，数字や空白を削除する．<br>
意味の重要度が高い可能性がある"名詞", "動詞", "形容詞"のみを使う．

In [17]:
# 形態素解析器の初期化
tagger = MeCab.Tagger(ipadic.MECAB_ARGS)
tagger.parse("")  # MeCabの初期化（空文字列を解析することで初期化）

'EOS\n'

In [18]:
# ストップワードの読み込み
# "stop_words.txt"をパス指定する
with open("stop_words.txt", encoding="utf-8") as f:
    stop_words = set(line.strip().lower() for line in f if line.strip())

# 英数字・記号などの正規化関数
def clean_token(surface):
    # 英字を小文字に正規化
    surface = surface.lower()
    # 数字は削除（全角・漢数字も含む）
    surface = re.sub(r'[0-9０-９一二三四五六七八九十百千万億兆]+', '', surface)
    # 記号・空白・改行を除去
    surface = re.sub(r'[^\wぁ-んァ-ン一-龥ー]+', '', surface)
    return surface

# トークナイズ＋クレンジング
def tokenize_clean(text):
    node = tagger.parseToNode(text)
    tokens = []
    while node:
        surface = node.surface
        features = node.feature.split(',')
        # 品詞取得
        pos = features[0]
        # 原形取得
        if len(features) > 6 and features[6] != '*':
            base = features[6]
        else:
            base = surface
        norm = clean_token(base)
        # 指定する品詞（今回は"名詞", "動詞", "形容詞"）に含まれる
        if pos in ["名詞", "動詞", "形容詞"]:
            # norm が削除されず存在している
            if norm:
                # norm がstop_words に含まれていない
                if norm not in stop_words:
                    tokens.append(norm)
        node = node.next
    return tokens

In [19]:
# トークナイズを含めた前処理の実行
sentences = df["body"].apply(tokenize_clean)
# ストップワード除去前後の結果を表示
print("ストップワード除去前：")
print(before_sentence)
print("ストップワード除去後：")
print(sentences[0])

ストップワード除去前：
['旧式', 'Mac', 'で', '禁断', 'の', 'パワーアップ', '！', '最新', 'PC', 'や', 'ソフト', 'を', '一挙', 'に', 'チェック', '【', 'IT', 'フラッシュ', 'バック', '】', 'テレビ', 'や', 'Twitter', 'と', '連携', 'できる', 'パソコン', 'や', '、', 'プロセッサ', 'や', '切り替わる', 'パソコン', 'など', '、', '面白い', 'パソコン', 'が', '次', 'から', '次', 'へ', 'と', '登場', 'し', 'た', '。', '旧式', 'Mac', 'の', '禁断', 'と', 'も', 'いえる', 'パワーアップ', '方法', 'から', '、', 'NEC', 'の', '最新', 'PC', '、', '話題', 'の', 'ThinkPad', 'X', '1', 'Hybrid', '、', '新', 'セキュリティ', 'ソフト', 'まで', '一挙', 'に', '紹介', 'しよ', 'う', '。', '■', 'インテル', 'SSD', '520', 'を', 'Mac', 'に', '装着', '！', '旧式', 'Mac', 'は', 'どれ', 'ほど', '高速', '化', 'する', 'の', 'か', '(', '上', ')', 'インテル', 'が', '最新', 'SSD', '「', '520', 'シリーズ', '」', 'を', '発売', 'し', 'た', '。', '現行', 'SSD', 'の', '中', 'でも', 'トップクラス', 'の', '性能', 'を', '誇る', '同', '製品', 'を', '、', '旧型', 'Mac', 'の', '高速', '化', 'を', '図る', 'という', 'ポイント', 'で', 'レビュー', 'し', 'て', 'み', 'た', '。', '少し', '風変わり', 'な', 'レビュー', 'と', 'なる', 'が', '、', 'どの', '程度', 'の', '効果', 'が', 'ある', 'か', '、', '期待', '大', 'で', 'あ

### 前処理したsentencesでWord2Vecを作成

In [ ]:
model = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=1,
    sg=0,
    seed=42
)

In [21]:
# 任意の単語と類似度が高い単語トップ10を表示
print(f"\"{target_word}\"と類似度の高い単語トップ10")
print("単語\t :類似度(最大値1, 最小値-1)")
for ranking in model.wv.most_similar(target_word, topn=10):
    print(f"{ranking[0]}\t :{ranking[1]}")

"スマホ"と類似度の高い単語トップ10
単語	 :類似度(最大値1, 最小値-1)
sii	 :0.9406098127365112
無敵	 :0.9398279786109924
切れ	 :0.9389480948448181
galaxy	 :0.9374306797981262
ケース	 :0.9345743060112
note	 :0.9304472208023071
物	 :0.9297348260879517
モバイルルーター	 :0.9274014234542847
ドコモ	 :0.9227121472358704
倒せる	 :0.9219352006912231


In [22]:
# 任意の2単語の類似度を表示
print(f"\"{target_word}\"と\"{target_word2}\"の類似度")
print(model.wv.similarity(target_word, target_word2))

"スマホ"と"難しい"の類似度
0.49308023


In [23]:
# ベクトルの差を計算
print(f"\"{target_word}\"-\"{target_word2}\"と類似度の高い単語トップ10")
print("単語\t :類似度(最大値1, 最小値-1)")
for ranking in model.wv.most_similar(positive=[target_word], negative=[target_word2]):
    print(f"{ranking[0]}\t :{ranking[1]}")

"スマホ"-"難しい"と類似度の高い単語トップ10
単語	 :類似度(最大値1, 最小値-1)
siii	 :0.6580267548561096
フレキシビリティ	 :0.6459165215492249
slider	 :0.6434098482131958
universal	 :0.6342478394508362
ul	 :0.6110766530036926
future	 :0.6052806973457336
swu	 :0.5860779285430908
stick	 :0.5829940438270569
スマートファミリンク	 :0.5728073120117188
悩ます	 :0.5723112225532532


## ストップワード除去前と後で単語ベクトルの結果の違いを確認
### 問8 学習済みモデルを読み込み

In [24]:
simple_model = Word2Vec.load("./model/simple_model.model")

In [25]:
# ストップワードの有無による語彙数の確認
print(f"ストップワード除去前の語彙数：{len(simple_model.wv)}")
print(f"ストップワード除去後の語彙数：{len(model.wv)}")

ストップワード除去前の語彙数：19717
ストップワード除去後の語彙数：15561


In [26]:
# 任意の単語と類似度が高い単語トップ10を表示
print(f"\"{target_word}\"と類似度の高い単語トップ10")
print("単語\t :類似度(最大値1, 最小値-1)")
print("ストップワード除去前：")
for ranking in simple_model.wv.most_similar(target_word, topn=10):
    print(f"{ranking[0]}\t :{ranking[1]}")
print("")
print("ストップワード除去後：")
for ranking in model.wv.most_similar(target_word, topn=10):
    print(f"{ranking[0]}\t :{ranking[1]}")

"スマホ"と類似度の高い単語トップ10
単語	 :類似度(最大値1, 最小値-1)
ストップワード除去前：
強い	 :0.8931419253349304
味方	 :0.8449782133102417
兵器	 :0.8356143236160278
スマ	 :0.8265565037727356
スタンド	 :0.8244155645370483
ケース	 :0.8136326670646667
潜入	 :0.8062107563018799
驚愕	 :0.8046680688858032
最新	 :0.7998527884483337
SIII	 :0.7960683107376099

ストップワード除去後：
sii	 :0.9406098127365112
無敵	 :0.9398279786109924
切れ	 :0.9389480948448181
galaxy	 :0.9374306797981262
ケース	 :0.9345743060112
note	 :0.9304472208023071
物	 :0.9297348260879517
モバイルルーター	 :0.9274014234542847
ドコモ	 :0.9227121472358704
倒せる	 :0.9219352006912231


In [27]:
# 任意の2単語の類似度を表示
print(f"\"{target_word}\"と\"{target_word2}\"の類似度")
print("ストップワード除去前：")
print(simple_model.wv.similarity(target_word, target_word2))
print("ストップワード除去後：")
print(model.wv.similarity(target_word, target_word2))

"スマホ"と"難しい"の類似度
ストップワード除去前：
0.32267633
ストップワード除去後：
0.49308023


In [28]:
# ベクトルの差を計算
print(f"\"{target_word}\"-\"{target_word2}\"と類似度の高い単語トップ10")
print("単語\t :類似度(最大値1, 最小値-1)")
print("ストップワード除去前：")
for ranking in simple_model.wv.most_similar(positive=[target_word], negative=[target_word2]):
    print(f"{ranking[0]}\t :{ranking[1]}")
print("")
print("ストップワード除去後：")
for ranking in model.wv.most_similar(positive=[target_word], negative=[target_word2]):
    print(f"{ranking[0]}\t :{ranking[1]}")

"スマホ"-"難しい"と類似度の高い単語トップ10
単語	 :類似度(最大値1, 最小値-1)
ストップワード除去前：
エヌ・ティ・ティ	 :0.6373079419136047
ファーストイーサネット	 :0.5560752749443054
太刀打ち	 :0.540340006351471
モバイルルーター	 :0.5328551530838013
モニタ	 :0.5227583050727844
si	 :0.518841028213501
タテヨコサイズ	 :0.5143628716468811
HK	 :0.5040014982223511
盤面	 :0.4982357621192932
スマートファミリンク	 :0.4976561665534973

ストップワード除去後：
siii	 :0.6580267548561096
フレキシビリティ	 :0.6459165215492249
slider	 :0.6434098482131958
universal	 :0.6342478394508362
ul	 :0.6110766530036926
future	 :0.6052806973457336
swu	 :0.5860779285430908
stick	 :0.5829940438270569
スマートファミリンク	 :0.5728073120117188
悩ます	 :0.5723112225532532


## 考察問題
### 問9 内省的評価によって前処理前と後のモデルの精度を評価してください

回答欄  
<font color = "Red">**TAさんの判断基準**</font>：ベクトル演算の結果が答えと同じにはなりません，的外れなことを言わない限り全て正解にして下さい。  
回答例
- 任意の単語と類似度が高い単語トップ10を表示したとき，全体的に類似度が向上している．
- (target_wordとtarget_word2が似ている単語の場合)任意の2単語間における類似度が1に近づいている．
- (target_wordとtarget_word2が関係のない単語の場合)任意の2単語間における類似度が0に近づいている．
- (target_wordとtarget_word2が似ていない単語の場合)任意の2単語間における類似度が-1に近づいている．
- "target_word - target_word2"を見ると，前処理後の方がしっくりくる．

参考
- 英字の単語はスマートフォンの機種名(siii=GALAXY S III, slider=AQUOS PHONE Slider, ul=Xperia UL SOL など)
- "スマホ"-"難しい"について：(ストップワード除去後の)スマホの類似度上位には"通信"や"ドコモ"が含まれている．そして，"難しい"との差には"フレキシビリティ"や実際のスマホの機種名などが含まれている．ここから，<font color = "Red">単語空間が"抽象的な内容"から"具体的な内容"に遷移した可能性</font>がある．それにより，"具体的に使いやすいスマホ"としてスマートフォンの機種名が現れた可能性がある．

### 問10 Word2Vecを用いた自然言語処理において前処理が重要な理由を考察してください

回答欄  
**Word2Vecは，大量の単語が"共に現れるか"という分布仮説に基づくため．**  
そのため，同じ意味の単語は統一し，意味のない単語は除去する必要がある．それにより，分散表現の"意味の近さ"を正しく反映できるようになる．  
形態素解析をするのは，文を単語に分割することで単語単位での意味学習を行うため．  
正規化("Word2Vec" ⇒ "word2vec")をするのは，同一語を統一し学習効率の向上ができるから．  
意味が関係ない記号(! や ?)などを除去するのは，意味を持つ単語だけで学習するため．  
ストップワードを用いるのは，情報量の少ない単語を除いて分散表現を明瞭にできるから．  
活用語の原型化("食べた" ⇒ "食べる")をするのは，意味が同じ語を統一し学習効率の向上ができるから．

## 時間が余った場合
自分でtarget_wordを設定してみてください．<br>
学習に使われなかった単語をキーにすると以下のようなエラーが出ます．<br>
`KeyError: "Key '???' not present in vocabulary"`

In [29]:
# 任意の単語が語彙に存在するか確認(Trueなら存在， Falseなら不在)
"単語" in model.wv

True

In [30]:
target_word1 = "pc"
target_word2 = "it"

In [31]:
# 任意の単語と類似度が高い単語トップ10を表示
print(f"\"{target_word1}\"と類似度の高い単語トップ10")
print("単語\t :類似度(最大値1, 最小値-1)")
for ranking in model.wv.most_similar(target_word1, topn=10):
    print(f"{ranking[0]}\t :{ranking[1]}")

"pc"と類似度の高い単語トップ10
単語	 :類似度(最大値1, 最小値-1)
ノート	 :0.9262063503265381
オールインワン	 :0.9074139595031738
パソコン	 :0.8830227851867676
ハイブリッド	 :0.8744773268699646
レッツ	 :0.8723442554473877
スレート	 :0.8627504110336304
快適	 :0.8561405539512634
無音	 :0.851604700088501
くたびれる	 :0.8416075706481934
sx	 :0.8209204077720642


In [32]:
# 任意の2単語の類似度を表示
print(f"\"{target_word1}\"と\"{target_word2}\"の類似度")
print(model.wv.similarity(target_word, target_word2))

"pc"と"it"の類似度
0.31691536


In [33]:
# ベクトルの差を計算
print(f"\"{target_word1}\"-\"{target_word2}\"と類似度の高い単語トップ10")
print("単語\t :類似度(最大値1, 最小値-1)")
for ranking in model.wv.most_similar(positive=[target_word1], negative=[target_word2]):
    print(f"{ranking[0]}\t :{ranking[1]}")

"pc"-"it"と類似度の高い単語トップ10
単語	 :類似度(最大値1, 最小値-1)
mdmi	 :0.6326950192451477
wuxga	 :0.6042160391807556
syncup	 :0.585832417011261
ストレート	 :0.5857669711112976
diskdropbox	 :0.583375096321106
チャージャー	 :0.5808027982711792
代替え	 :0.5731554627418518
プラスーマイナス	 :0.572584331035614
延ばす	 :0.5649582743644714
非力	 :0.5631615519523621
